# **Red neuronal convolucional para el diagnóstico de nódulos tiroideos según la clasificación EU-TIRADS**

## Por Alejandro Martínez Hernández

### Notebook 2/3

# **Procesamiento de datos**

Ya conociendo la estructura de los datos, estos deben ser preparados y nuevamente organizados para poder usarlos en algún entrenamiento de un modelo. 


## Creación de directorios

En la carpeta "organized" se crean otras carpetas para almacenar las imágenes (\images) y anotaciones (\notes) correspondientes que se van a usar para el entrenamiento.

Dentro de estas carpetas habrán otras que contendrán las imágenes y anotaciones crudas (\Raw) y otra para las imagenes y anotaciones ya procesadas (cropped).

In [5]:
import os
import shutil

def crear_subcarpetas(carpeta_principal, subcarpetas, subcarpeta_padre=None):
    """
    Crea subcarpetas dentro de una carpeta principal. Si se especifica una subcarpeta_padre,
    las subcarpetas se crearán dentro de esta.
    
    Parámetros:
    carpeta_principal (str): Ruta de la carpeta principal.
    subcarpetas (list): Lista de nombres de subcarpetas a crear.
    subcarpeta_padre (str): Nombre de la subcarpeta dentro de la cual se crearán nuevas subcarpetas.
    """
    # Determinar la ruta base donde se crearán las subcarpetas
    ruta_base = carpeta_principal if subcarpeta_padre is None else os.path.join(carpeta_principal, subcarpeta_padre)
    
    # Asegurar que la ruta base exista
    if not os.path.exists(ruta_base):
        os.makedirs(ruta_base)
    
    # Crear cada subcarpeta
    for subcarpeta in subcarpetas:
        os.makedirs(os.path.join(ruta_base, subcarpeta), exist_ok=True)

# Aplicación de función
carpeta_principal_2 = 'db_unal'
subcarpetas_dentro_v2 = ['images', 'notes']  # Subcarpetas a crear directamente en db_unal/organized
carpeta_principal_3 = 'db_unal/organized'
subcarpetas_dentro_v3 = ['raw', 'cropped']  # Subcarpetas a crear directamente en db_unal/organized

# Crear subcarpetas adicionales dentro de db_unal/organized
crear_subcarpetas(carpeta_principal_2, subcarpetas_dentro_v2, 'organized')
crear_subcarpetas(carpeta_principal_3, subcarpetas_dentro_v3, 'images')
crear_subcarpetas(carpeta_principal_3, subcarpetas_dentro_v3, 'notes')

## Importar los datos

Se trae el dataframe *df_actualizado.csv* que se generó en el notebook *exploracion_datos.ipynb*

In [6]:
import pandas as pd
import os

def cargar_csv_de_forma_segura(ruta_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas de manera segura.
    
    Parámetros:
    ruta_archivo (str): La ruta completa al archivo CSV que se desea cargar.
    
    Retorna:
    DataFrame: Un DataFrame de pandas con los datos cargados desde el archivo CSV.
    None: Retorna None si ocurre un error durante la carga del archivo.
    """
    # Verificar si el archivo existe
    if not os.path.exists(ruta_archivo):
        print(f"El archivo {ruta_archivo} no existe.")
        return None
    
    try:
        # Usar pandas para leer el archivo CSV
        df = pd.read_csv(ruta_archivo)
        return df
    except pd.errors.EmptyDataError:
        print("El archivo está vacío.")
    except pd.errors.ParserError:
        print("Error al parsear el archivo CSV.")
    except Exception as e:
        print(f"Error al cargar el archivo: {e}")
    
    return None

# Cargar el archivo
ruta_archivo = 'df_actualizado.csv'
df_actualizado = cargar_csv_de_forma_segura(ruta_archivo)

# Mostar dataframe
df_actualizado



,Numero paciente,TIRADS,Segmentacion,Numero,Edad,Sexo,Composicion,Ecogenicidad,Margenes,Calcificaciones
0,1_1,5.0,YES,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1_2,5.0,YES,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2_1,4.0,YES,2,49.0,F,solid,hyperechogenicity,well defined,non
3,3_1,4.0,YES,3,31.0,F,spongiform,isoechogenicity,well defined,microcalcifications
4,4_1,3.0,YES,4,37.0,F,spongiform,isoechogenicity,well defined,microcalcifications
...,...,...,...,...,...,...,...,...,...,...
167,146_1,3.0,YES,146,NaN,NaN,NaN,NaN,NaN,NaN
168,147_1,5.0,YES,147,NaN,NaN,NaN,NaN,NaN,NaN
169,148_1,4.0,YES,148,37.0,F,solid,hypoechogenicity,well defined,non
170,149_1,5.0,YES,149,78.0,M,NaN,NaN,NaN,NaN


## Copiar datos a tratar

Ya que importamos la base de datos que nos indica que imágenes y anotaciones vamos a usar, se generará una copia de cada uno de estos a las carpetas \raw dentro de \organized.

In [7]:
import os
import shutil

def copiar_archivos_relevantes(df, columna_id, directorio_origen, directorio_destino, extension):
    """
    Copia archivos especificados en un DataFrame desde un directorio origen a un directorio destino.
    
    Parámetros:
    df (DataFrame): DataFrame que contiene los identificadores de los archivos.
    columna_id (str): Nombre de la columna en el DataFrame que contiene los identificadores de los archivos.
    directorio_origen (str): Ruta del directorio donde se encuentran los archivos originales.
    directorio_destino (str): Ruta del directorio destino donde se copiarán los archivos.
    extension (str): Extensión de los archivos a copiar.
    """
    # Asegurarse de que el directorio destino existe, si no, crearlo
    os.makedirs(directorio_destino, exist_ok=True)
    
    # Iterar sobre cada identificador de archivo en el DataFrame
    for _, fila in df.iterrows():
        archivo = f"{fila[columna_id]}{extension}"
        ruta_origen = os.path.join(directorio_origen, archivo)
        ruta_destino = os.path.join(directorio_destino, archivo)
        
        # Copiar el archivo si existe en el directorio origen
        if os.path.exists(ruta_origen):
            shutil.copy(ruta_origen, ruta_destino)
            print(f"Copiado: {archivo}")
        else:
            print(f"No existe: {archivo}")

# Rutas de los directorios origen
directorio_imagenes = 'db_unal/originals/DDTI_V1'
directorio_anotaciones = 'db_unal/originals/DDTI_V2/RadiologistSegmentations'

# Rutas de los directorios destino
destino_imagenes = 'db_unal/organized/images/raw'
destino_anotaciones = 'db_unal/organized/notes/raw'

# Copiar imágenes y anotaciones
copiar_archivos_relevantes(df_actualizado, 'Numero paciente', directorio_imagenes, destino_imagenes, '.jpg')
copiar_archivos_relevantes(df_actualizado, 'Numero paciente', directorio_anotaciones, destino_anotaciones, '.nii')
print('Proceso finalizado. Por favor revise la carpeta')


Copiado: 1_1.jpg
Copiado: 1_2.jpg
Copiado: 2_1.jpg
Copiado: 3_1.jpg
Copiado: 4_1.jpg
Copiado: 5_1.jpg
Copiado: 6_1.jpg
Copiado: 7_1.jpg
Copiado: 8_1.jpg
Copiado: 9_1.jpg
Copiado: 10_1.jpg
Copiado: 11_1.jpg
Copiado: 12_1.jpg
Copiado: 13_1.jpg
Copiado: 14_1.jpg
Copiado: 15_1.jpg
Copiado: 16_1.jpg
Copiado: 17_1.jpg
Copiado: 18_1.jpg
Copiado: 19_1.jpg
Copiado: 20_1.jpg
Copiado: 21_1.jpg
Copiado: 22_1.jpg
Copiado: 23_1.jpg
Copiado: 24_1.jpg
Copiado: 25_1.jpg
Copiado: 26_1.jpg
Copiado: 27_1.jpg
Copiado: 28_1.jpg
Copiado: 29_1.jpg
Copiado: 30_1.jpg
Copiado: 31_1.jpg
Copiado: 32_1.jpg
Copiado: 33_1.jpg
Copiado: 34_1.jpg
Copiado: 35_1.jpg
Copiado: 36_1.jpg
Copiado: 38_1.jpg
Copiado: 39_1.jpg
Copiado: 40_1.jpg
Copiado: 41_1.jpg
Copiado: 42_1.jpg
Copiado: 43_1.jpg
Copiado: 44_1.jpg
Copiado: 45_1.jpg
Copiado: 46_1.jpg
Copiado: 47_1.jpg
Copiado: 48_1.jpg
Copiado: 49_1.jpg
Copiado: 50_1.jpg
Copiado: 51_1.jpg
Copiado: 52_1.jpg
Copiado: 53_1.jpg
Copiado: 54_1.jpg
Copiado: 55_1.jpg
Copiado: 56_1.jpg
Co

## Código para hacer recorte

In [11]:
import cv2
import os
from pathlib import Path

# Definir la ruta base del proyecto
base_path = Path.cwd().parent

# Funciones de ajuste de niveles y recorte de imagen
def ajustar_niveles(imagen, nivel_min, nivel_max):
    """
    Ajusta los niveles de la imagen utilizando cv2.normalize.
    """
    return cv2.normalize(imagen, None, alpha=nivel_min, beta=nivel_max, norm_type=cv2.NORM_MINMAX)

def encontrar_y_recortar_imagen(input_path, output_path):
    """
    Encuentra la imagen de ultrasonido dentro de una captura de pantalla y la recorta
    sin modificar el brillo y el contraste en la imagen recortada.
    """
    # Leer la imagen original
    imagen_original = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
    
    # Convertir la imagen a escala de grises
    imagen_gris = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2GRAY)
    
    # Ajustar los niveles de la imagen para mejorar el contraste
    imagen_contrastada = ajustar_niveles(imagen_gris, 0, 255)
    
    # Umbralizar la imagen para obtener una binarización
    _, imagen_binaria = cv2.threshold(imagen_contrastada, 1, 255, cv2.THRESH_BINARY)
    
    # Encontrar los contornos en la imagen binaria
    contornos, _ = cv2.findContours(imagen_binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Procesar cada contorno encontrado
    for contorno in contornos:
        # Calcular el rectángulo delimitador para el contorno
        x, y, ancho, alto = cv2.boundingRect(contorno)
        borde_mas_corto = min(ancho, alto)
        borde_mas_largo = max(ancho, alto)
        proporcion = borde_mas_largo / borde_mas_corto

        # Comprobar si el contorno cumple con las condiciones de tamaño mínimo y proporción
        if borde_mas_corto >= 250 and proporcion <= 1.5:
            # Recortar la imagen original (sin ajustes de contraste) según el contorno que cumple con los requisitos
            imagen_recortada = imagen_original[y:y+alto, x:x+ancho]
            
            # Guardar la imagen recortada
            cv2.imwrite(output_path, imagen_recortada)
            break  # Terminar después de procesar el primer contorno válido

# Rutas de las carpetas de imágenes usando rutas relativas
input_folder1 = 'db_unal/organized/images/raw'
output_folder1 = 'db_unal/organized/images/cropped'

# Función para procesar las imágenes
def ubicar(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        try:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                input_path = input_folder / filename
                base_name = filename.rsplit('.', 1)[0] + '.png'  # Cambiar la extensión a .png
                output_path = output_folder / base_name
                
                try:
                    encontrar_y_recortar_imagen(str(input_path), str(output_path))
                except Exception as e:
                    print(f"No se pudo procesar el archivo {filename}: {e}")
        except Exception as e:
            print(f"No se pudo procesar el archivo {filename}: {e}")

# Procesar imágenes
ubicar(input_folder1, output_folder1)

No se pudo procesar el archivo 100_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 101_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 102_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 103_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 104_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 105_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 106_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 107_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 108_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 109_1.jpg: unsupported operand type(s) for /: 'str' and 'str'
No se pudo procesar el archivo 10_1.jpg: unsupported operand type(s) f

## Código para hacer undersampling

##  Aumento de datos

Dejaré este código para volver a usarlo en un futuro

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Definir la ruta base del proyecto
base_path = Path.cwd().parent

# Ubicación de las imágenes ordenadas (usando pathlib para construir la ruta)
database = base_path / 'data' / 'us_images' / 'procesadas' / 'ordenadas'

# Instanciar ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=15,
    zoom_range=[0.5, 1.5],
    validation_split=0.2
)

# Generadores para sets de entrenamiento y validación
data_gen_entrenamiento = datagen.flow_from_directory(
    database,
    target_size=(300, 300),
    batch_size=32,
    shuffle=True,
    subset='training'
)

data_gen_pruebas = datagen.flow_from_directory(
    database,
    target_size=(300, 300),
    batch_size=32,
    shuffle=True,
    subset='validation'
)

# Imprimir 10 imágenes del generador de entrenamiento
for imagen, etiqueta in data_gen_entrenamiento:
    for i in range(10):
        plt.subplot(2, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(imagen[i])
    break
plt.show()